In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/warehouse'

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL Hive integration example") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

In [2]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db|globalinternetusa...|      false|
| tabd_db|globalsocialnetworks|      false|
| tabd_db|       internetusage|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|  socialnetworkusers|      false|
+--------+--------------------+-----------+



In [3]:
spark.sql(
    """
    DROP TABLE IF EXISTS tabd_db.SocialMediaUsageForEnterprisesLevelDevelopment
    """
)

spark.sql(
    """
    CREATE EXTERNAL TABLE tabd_db.SocialMediaUsageForEnterprisesLevelDevelopment (
        country VARCHAR(45),
        percentage FLOAT,
        purpose VARCHAR(45),
        enterprise_size VARCHAR(45),
        internet_users_percentage INT,
        status INT
    )
    STORED AS PARQUET
    LOCATION 'hdfs://hdfs-nn:9000/warehouse/tabd.db/SocialMediaUsageForEnterprisesLevelDevelopment/'
    """
)

# tblproperties('skip.header.line.count'='1')
# can be used for csvs with header
# but spark sql cannot understand that at the moment, while reading the data using sql queries
# so just when creating hive tables backed up by csvs, avoid headers
# in the project we wont use hive text tables, so all good.

DataFrame[]

In [4]:
spark.sql(
    """
    SHOW TABLES FROM tabd_db
    """
).show()

+--------+--------------------+-----------+
|database|           tableName|isTemporary|
+--------+--------------------+-----------+
| tabd_db|globalinternetusa...|      false|
| tabd_db|globalsocialnetworks|      false|
| tabd_db|       internetusage|      false|
| tabd_db|numberofworldwide...|      false|
| tabd_db|socialmediausagef...|      false|
| tabd_db|  socialnetworkusers|      false|
+--------+--------------------+-----------+



In [5]:
# Let's look into HDFS

In [6]:
spark.sql(
    """
    SELECT *
    FROM tabd_db.SocialMediaUsageForEnterprisesLevelDevelopment
    """
).show()

+-------+----------+-------+---------------+-------------------------+------+
|country|percentage|purpose|enterprise_size|internet_users_percentage|status|
+-------+----------+-------+---------------+-------------------------+------+
+-------+----------+-------+---------------+-------------------------+------+



In [7]:
spark.sql(
    """
    DESCRIBE FORMATTED tabd_db.SocialMediaUsageForEnterprisesLevelDevelopment
    """
).toPandas()

,col_name,data_type,comment
0,country,varchar(45),None
1,percentage,float,None
2,purpose,varchar(45),None
3,enterprise_size,varchar(45),None
4,internet_users_percentage,int,None
5,status,int,None
6,,,
7,# Detailed Table Information,,
8,Database,tabd_db,
9,Table,socialmediausageforenterprisesleveldevelopment,


In [8]:
# Let's put the files into HDFS

In [9]:
spark.sql(
    """
    SELECT *
    FROM tabd_db.SocialMediaUsageForEnterprisesLevelDevelopment
    """
).toPandas()

,country,percentage,purpose,enterprise_size,internet_users_percentage,status


In [10]:

spark.sql(
    """
    SELECT *
    FROM tabd_db.SocialMediaUsageForEnterprisesLevelDevelopment
    """
).show()

+-------+----------+-------+---------------+-------------------------+------+
|country|percentage|purpose|enterprise_size|internet_users_percentage|status|
+-------+----------+-------+---------------+-------------------------+------+
+-------+----------+-------+---------------+-------------------------+------+



In [11]:
spark.stop()